In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir('/content/drive/MyDrive')


Mounted at /content/drive/


In [ ]:
cd CZ4034/

/content/drive/MyDrive/CZ4034


In [ ]:
import pandas as pd

pd.options.display.max_colwidth = 10000

train_df=pd.read_csv('shared_train_set.csv')
test_df=pd.read_csv('shared_test_set.csv')

train_df.head()
test_df.head()

text  label
0                                                                                                                                                                                      Itâs good as a first job but as a person who pays the rent, it just ainât happening. They keep hiring more people and cutting down my hours when they know I need the money, which is inconsiderate.      0
1                                                                                                                                                                                                                                 i enjoyed working here, teamwork environment,lots of help, nice balance of work /family life.good pto, medical,401k benefits. steady raises, no glass ceiling      1
2                                                                                                                                                                                                                                    Management always worked with my schedule. I worked hard so that I could earn the flexibility in my schedule and not just expect it from store management.      0
3  Work-life balanceGood group of people to work with. Respectful and friendly.Pay & benefitsGood health insurance and 401kJob security and advancementIf you are hardworking and you are honest you will have a job!OverallRetired after 35+ years with this great company! Would not have been able to retire without the pay and benefits this company offers. It's hard work but rewarding!      1
4                                                                                                                                                                                                                             The good about working at stanley black and decker is, the consistent team work and working ethic. Now as to anything bad, I have never experienced that as such.      1

In [ ]:
import numpy as np
import regex as re
import nltk
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [ ]:
from sklearn.utils import shuffle
# select two cols, rename, get rid of neutral
train_df = train_df[train_df.label != 0]
test_df = test_df[test_df.label != 0]

train_df = shuffle(train_df, random_state=1000)

train_df.to_csv('shared_data_train_pol.csv')
test_df.to_csv('shared_data_test_pol.csv')

print(len(train_df[train_df.label == 1]))
print(len(train_df[train_df.label == 2]))
print(len(test_df[test_df.label == 1]))
print(len(test_df[test_df.label == 2]))

422
377
94
94


In [ ]:
import re

def alphanum_cleaning(x):
    x = str(x)
    pattern = '[^a-zA-Z\ ]'
    x = re.sub(pattern,'',x)
    x = x.lower()
    x = x.strip()
    return x

train_df['text'] = train_df['text'].apply(alphanum_cleaning)
test_df['text'] = test_df['text'].apply(alphanum_cleaning)

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def stemming(x):
    stemmer = PorterStemmer()
    x = word_tokenize(x)
    store = ''
    
    for i in x:
        store += stemmer.stem(i) + ' '
        
    return store

train_df['stemmed_text'] = train_df['text'].apply(stemming)
test_df['stemmed_text'] = test_df['text'].apply(stemming)

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.stem import WordNetLemmatizer
def lemmatization(corpus):
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]
  
train_df['lemmatized_text'] = lemmatization(train_df['text'])
test_df['lemmatized_text'] = lemmatization(test_df['text'])

In [ ]:
# Get english stopwords
stop = stopwords.words('english')
additional_stopwords = ["'s","...","'ve","``","''","'m",'--',"'ll","'d"]
stop = set(stop + additional_stopwords)
def remove_stop(x):
    x = word_tokenize(x)
    store = ''
    
    for i in x:
        if i not in stop:
            store += i + ' '
            
    return store
  
train_df['stemmed_text_clean'] = train_df['stemmed_text'].apply(remove_stop)
train_df['lemmatized_text_clean'] = train_df['lemmatized_text'].apply(remove_stop)
train_df['text_cleaned'] = train_df['text'].apply(remove_stop)

test_df['stemmed_text_clean'] = test_df['stemmed_text'].apply(remove_stop)
test_df['lemmatized_text_clean'] = test_df['lemmatized_text'].apply(remove_stop)
test_df['text_cleaned'] = test_df['text'].apply(remove_stop)


In [ ]:
from sklearn.model_selection import train_test_split, KFold
from joblib import dump, load
from time import time

col_selected = 'text'


x_train, y_train = train_df[col_selected], train_df['label']
x_test, y_test = test_df[col_selected], test_df['label']

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,2))



comment_matrix = vectorizer.fit_transform(x_train)

from sklearn.naive_bayes import MultinomialNB
comment_classifier = MultinomialNB().fit(comment_matrix, y_train)

from sklearn import metrics
start = time()
test_vector = vectorizer.transform(x_test)
result = comment_classifier.predict(test_vector)
end = time()
confuse = metrics.confusion_matrix(y_test, result)

# for stack ensemble
pred_x = comment_classifier.predict(comment_matrix) 
pd.DataFrame(pred_x).to_csv('bayes_pred_train_pol.csv')
pd.DataFrame(result).to_csv('bayes_pred_test_pol.csv')

from sklearn.metrics import accuracy_score

F1_score = metrics.f1_score(y_test, result)
average_precision = metrics.average_precision_score(y_test, result)
precision_score= metrics.precision_score(y_test, result)
recall_score = metrics.recall_score(y_test, result)

print(f'=========={col_selected}==========')
print(f'accuracy: {accuracy_score(y_test, result)}')
print('F1 score: {0:0.3f}'.format(F1_score))
print('Precision score: {0:0.3f}'.format(precision_score))
print('Recall score: {0:0.3f}'.format(recall_score))
print('Average precision-recall score: {0:0.3f}'.format(
      average_precision))
print(f"Runtime of the program is {end - start}, number of record classified is {test_vector.shape[0]} \naverage number of record per second is {test_vector.shape[0]/(end-start)}")

dump(comment_classifier, 'polarity_naive.joblib') 

accuracy: 0.925531914893617
==========text==========
accuracy: 0.925531914893617
F1 score: 0.922
Precision score: 0.965
Recall score: 0.883
Average precision-recall score: 0.454
Runtime of the program is 0.018883705139160156, number of record classified is 188 
average number of record per second is 9955.673349830817


In [ ]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
nfolds=5

Cs = [0.001, 0.01, 0.1, 1, 10]
gammas = [0.05, 0.1, 0.15, 0.20, 0.25]
degrees = [0, 1, 2, 3, 4, 5, 6]
kernels = ['rbf', 'linear', 'poly']
param_grid = {'C': Cs, 'gamma' : gammas, 'degree' : degrees, 'kernel' : kernels}

grid_search = GridSearchCV(svm.SVC(), param_grid, cv = nfolds)
grid_search.fit(comment_matrix, y_train)
grid_search.best_params_


{'C': 0.1, 'degree': 0, 'gamma': 0.05, 'kernel': 'linear'}

In [ ]:
from sklearn import svm
best_params_ = {'C': 0.1, 'degree': 0, 'gamma': 0.05, 'kernel': 'linear'}
SVM = svm.SVC(**best_params_).fit(comment_matrix, y_train)

start = time()
test_vector = vectorizer.transform(x_test)
result = SVM.predict(test_vector)
end = time()
confuse = metrics.confusion_matrix(y_test, result)

# for stack ensemble
pred_x = SVM.predict(comment_matrix) 
pd.DataFrame(pred_x).to_csv('svm_pred_train_pol.csv')
pd.DataFrame(result).to_csv('svm_pred_test_pol.csv')



F1_score = metrics.f1_score(y_test, result)
average_precision = metrics.average_precision_score(y_test, result)
precision_score= metrics.precision_score(y_test, result)
recall_score = metrics.recall_score(y_test, result)

print(f'=========={col_selected}==========')
print(f'accuracy: {accuracy_score(y_test, result)}')
print('F1 score: {0:0.3f}'.format(F1_score))
print('Precision score: {0:0.3f}'.format(precision_score))
print('Recall score: {0:0.3f}'.format(recall_score))
print('Average precision-recall score: {0:0.3f}'.format(
      average_precision))
print(f"Runtime of the program is {end - start}, number of record classified is {test_vector.shape[0]} \naverage number of record per second is {test_vector.shape[0]/(end-start)}")

accuracy: 0.9148936170212766
==========text==========
accuracy: 0.9148936170212766
F1 score: 0.916
Precision score: 0.906
Recall score: 0.926
Average precision-recall score: 0.468
Runtime of the program is 0.07013559341430664, number of record classified is 188 
average number of record per second is 2680.521983886868


In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

# A parameter grid for XGBoost
# can add more range
params = {
    'eta': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.5, 0.7, 1.0],
    'colsample_bytree': [0.5, 0.7, 1.0],
    'min_child_weight': [1, 5, 10],
}

xgb = XGBClassifier()

folds = 5

grid = GridSearchCV(estimator=xgb, param_grid=params, scoring='accuracy', cv=folds, verbose=3)
grid.fit(comment_matrix, y_train)


Fitting 3 folds for each of 243 candidates, totalling 729 fits
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5, score=0.854, total=   1.8s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s


[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5, score=0.861, total=   1.9s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.8s remaining:    0.0s


[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.5, score=0.827, total=   1.8s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7 
[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7, score=0.873, total=   1.8s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7 
[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7, score=0.868, total=   1.7s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7 
[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=0.7, score=0.850, total=   1.9s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=1.0 
[CV]  colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=1.0, score=0.861, total=   1.9s
[CV] colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=1.0 
[CV]  cols

[Parallel(n_jobs=1)]: Done 729 out of 729 | elapsed: 44.3min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'colsample_bytree': [0.5, 0.7, 1.0],
                         'eta': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7],
                         'min_child_weigh

In [ ]:
print('\n Best estimator:')
print(grid.best_estimator_)
print('\n Best parameters:')
print(grid.best_params_)
results = pd.DataFrame(grid.cv_results_)
results.to_csv('xgb-grid-search-results.csv', index=False)


 Best estimator:
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, eta=0.01, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1.0, verbosity=1)

 Best parameters:
{'colsample_bytree': 0.5, 'eta': 0.01, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 1.0}


In [ ]:


from xgboost import XGBClassifier
xgb = XGBClassifier(colsample_bytree=0.5, eta=0.01, max_depth=3, min_child_weight=1, subsample=1.0)
xgb.fit(comment_matrix, y_train)
start = time()
test_vector = vectorizer.transform(x_test)
result = xgb.predict(test_vector)
end = time()
confuse = metrics.confusion_matrix(y_test, result)

# for stack ensemble
pred_x = xgb.predict(comment_matrix) 
pd.DataFrame(pred_x).to_csv('xgb_pred_train_pol.csv')
pd.DataFrame(result).to_csv('xgb_pred_test_pol.csv')


F1_score = metrics.f1_score(y_test, result)
average_precision = metrics.average_precision_score(y_test, result)
precision_score= metrics.precision_score(y_test, result)
recall_score = metrics.recall_score(y_test, result)

print(f'=========={col_selected}==========')
print(f'accuracy: {accuracy_score(y_test, result)}')
print('F1 score: {0:0.3f}'.format(F1_score))
print('Precision score: {0:0.3f}'.format(precision_score))
print('Recall score: {0:0.3f}'.format(recall_score))
print('Average precision-recall score: {0:0.3f}'.format(
      average_precision))
print(f"Runtime of the program is {end - start}, number of record classified is {test_vector.shape[0]} \naverage number of record per second is {test_vector.shape[0]/(end-start)}")

accuracy: 0.8723404255319149
==========text==========
accuracy: 0.8723404255319149
F1 score: 0.874
Precision score: 0.865
Recall score: 0.883
Average precision-recall score: 0.455
Runtime of the program is 0.03658318519592285, number of record classified is 188 
average number of record per second is 5138.9729733252525
